<img align='left' src = https://linea.org.br/wp-content/themes/LIneA/imagens/logo-header.jpg width=150 style='padding: 20px'> 

# Tutorial: particionamento de dados com Hipscat  

Passo-a-passo para conversão de catálogos astronômicos para o formato Hipscat. 

Contato 1: Julia Gschwend ([julia@linea.org.br](mailto:julia@linea.org.br))

Contato 2: Luigi Silva ([luigi.silva@linea.org.br](mailto:luigi.silva@linea.org.br))
<br>
Última verificação: 09/05/2024
<br>



#### Acknowledgements

<font color='red'> texto pedindo citação - ver modelo no jupyterhub do dirac/lincc </font>

'_Esta pesquisa utilizou recursos computacionais da Associação Laboratório Interinstitucional de e-Astronomia (LIneA) com o apoio financeiro do INCT do e-Universo (Processo n.º 465376/2014-2)._' 

LIneA

LINCC Frameworks 


# Introdução

<font color='red'> conceito healpix </font>

<font color='red'> conceito hipscat </font>

<font color='red'> conceito x-match espacial </font>

<font color='red'> desafios técnicos, escalabilidade, objetos nas bordas, etc </font>



# Importação das bibliotecas e configurações

Requisitos para este notebook:

* **Bibliotecas essenciais**: hipscat, hipscat_import, lsdb.
* **Bibliotecas gerais**: numpy, tables_io, astropy. 
* **Bibliotecas de visualização**: bokeh, holoviews, geoviews, datashader, matplotlib.
* **Bibliotecas de manipulação de dados**: pandas, geopandas.
* **Bibliotecas para a obtenção de dados**: pzserver, dblinea.
* **Arquivo auxiliar**: [des-round19-poly.txt](https://github.com/kadrlica/skymap/blob/master/skymap/data/des-round19-poly.txt) (contorno da área coberta pelo levantamento do DES DR2, i.e., DES _footprint_, 2019 version).

Faça o download do arquivo `des-round19-poly.txt` do repositório [kadrlica/skymap](https://github.com/kadrlica/skymap) no GitHub:

In [ ]:
#! wget https://raw.githubusercontent.com/kadrlica/skymap/master/skymap/data/des-round19-poly.txt -O des-round19-poly.txt 

## Importações

In [ ]:
### GERAL
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import time

import matplotlib.pyplot as plt
import tables_io
import psutil
import sys
import math

from shapely.geometry import Polygon

### ASTROPY
from astropy.coordinates import SkyCoord
from astropy import units as u

# BOKEH
import bokeh
from bokeh.io import output_notebook, show, output_file, reset_output
from bokeh.models import ColumnDataSource, Range1d, HoverTool, CustomJSHover
from bokeh.models import CDSView, GroupFilter
from bokeh.plotting import figure, gridplot
from bokeh.transform import factor_cmap

# HOLOVIEWS
import holoviews as hv
from holoviews import streams, opts
from holoviews.operation import histogram
from holoviews.operation.datashader import datashade, rasterize, shade, dynspread, spread
from holoviews.plotting.util import process_cmap

# GEOVIEWS
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

### HIPSCAT
import hipscat_import.pipeline as runner
from hipscat_import.catalog.arguments import ImportArguments
from hipscat_import.catalog.file_readers import CsvReader

### PZ SERVER
from pzserver import PzServer 

# DATASHADER
import datashader as dsh

# DB LIneA
from dblinea import DBBase

Imprimindo as versões do Python, Numpy, Bokeh e Holoviews:

In [ ]:
print('Python version: ' + sys.version)
print('Bokeh version: ' + bokeh.__version__)
print('HoloViews version: ' + hv.__version__)
print("Datashader version: " + dsh.__version__)

## Configurações

### Configurações das bibliotecas

Definindo o número de linhas que o pandas irá exibir.

In [ ]:
pd.set_option('display.max_rows', 10)

Configurando o holoviews e o geoviews para trabalhar com o bokeh:

In [ ]:
hv.extension('bokeh')

In [ ]:
gv.extension('bokeh')

Configurando os plots do bokeh para serem em linha:

In [ ]:
output_notebook()

Configurando os plots do matplotlib para serem em linha:

In [ ]:
%matplotlib inline

Lendo o token e definindo a classe PzServer.

In [ ]:
with open('/home/luigi.silva/token.txt', 'r') as file:
    token = file.read().rstrip()
pz_server = PzServer(token=token, host="pz-dev") 

### Configurações dos plots de distribuição de objetos

Definindo os parâmetros gerais para os plots de distribuição de objetos utilizando geoviews e datashader.

In [ ]:
height = 500    ### Altura do plot.
width = 1000    ### Largura do plot.
padding = 0.05  ### Padding do plot.
xlabel = 'R.A.' ### Label para o eixo x.
ylabel = 'Dec.' ### Label para o eixo y.

Definindo os labels de R.A. e DEC. a serem exibidos nos plots de distribuição de objetos utilizando geoviews e datashader.

In [ ]:
longitudes = np.arange(30, 360, 30)
latitudes = np.arange(-75, 76, 15)

lon_labels = [f"{lon}°" for lon in longitudes]
lat_labels = [f"{lat}°" for lat in latitudes]

labels_data = {
    "lon": list(np.flip(longitudes)) + [-180] * len(latitudes),
    "lat": [0] * len(longitudes) + list(latitudes),
    "label": lon_labels + lat_labels,
}

df_labels = pd.DataFrame(labels_data)

labels_plot = gv.Labels(df_labels, kdims=["lon", "lat"], vdims=["label"]).opts(
    text_font_size="12pt",
    text_color="black",
    text_align='right',
    text_baseline='bottom',
    projection=crs.Mollweide()
)

Definindo o grid dos gráficos.

In [ ]:
grid = gf.grid()

# Caracterização da amostra de exemplo

## Lendo o footprint do DES

A seguir, vamos ler o footprint do DES DR2 e já aplicar uma conversão de coordenadas em R.A. e DEC.

In [ ]:
foot_ra, foot_dec = np.loadtxt('des-round19-poly.txt', unpack=True)
foot_coords = SkyCoord(ra=-foot_ra*u.degree, dec=foot_dec*u.degree, frame='icrs')
foot_df = pd.DataFrame({'foot_ra': np.array(foot_coords.ra.wrap_at(180*u.degree)), 
                        'foot_dec': np.array(foot_coords.dec)})

Já vamos definir também a curva do footprint do DES, para os plots de distribuição de objetos.

In [ ]:
ra_dec_foot = gv.Path((-foot_ra, foot_dec)).opts(line_width=3, color='orange')

## Dados espectroscópicos

### Amostra pública de *redshifts* espectroscópicos - Completa

O produto [Public spec-z compilation](https://pz-server-dev.linea.org.br/product/26_public_specz_compilation) é um compilado de catálogos de diferentes levantamentos os quais foram coletados ao longo dos anos de operação do Dark Energy Survey (DES) e agrupados sistematicamente pela ferramenta DES Science Portal (pipeline Spectroscopic Sample) para formar a base de um conjunto de treinamento para algoritmos de cálculo de *redshifts* fotométricos baseados em machine learning. 

In [ ]:
#specz = pz_server.get_product('26_public_specz_compilation')

In [ ]:
#specz.display_metadata()

In [ ]:
#specz.data.describe()

In [ ]:
#specz.plot()

### Amostra utilizada nos testes - Seleção do 2dflens

Como dito anteriormente, o produto [Public spec-z compilation](https://pz-server-dev.linea.org.br/product/26_public_specz_compilation) é um compilado de *redshifts* espectroscópicos de vários levantamentos astronômicos. Muitos desses levantamentos observaram regiões comuns do céu. Ao agrupar todas as medidas em um catálogo único, geralmente há medidas múltiplas de *redshift* espectroscópico para um mesmo objeto. Para identificar esses casos, o *pipeline* Spectroscopic Sample faz uma combinação espacial entre as coordenadas equatoriais de "todos contra todos" com um raio de busca de 1.0 *arcsec* de cada objeto. Então, ele aplica uma seleção para manter apenas uma medida para cada objeto extragaláctico presente na amostra, seguindo o critério abaixo para escolha e desempate:

1. medida com a maior _flag_ de qualidade (`flag_des`)
2. medida com o menor erro no *redshift* (`err_z`)
3. medida obtida pelo levantamento mais recente

Portanto, os objetos do levantamento 2dflens apresentados aqui são aqueles que permaneceram após a seleção considerando todos os catálogos presentes no produto [Public spec-z compilation](https://pz-server-dev.linea.org.br/product/26_public_specz_compilation). Um corte de qualidade também foi aplicado onde apenas objetos com flag_des ⩾ 3 foram incluídos na amostra.

#### Todos os objetos do 2dflens, contidos no produto Public spec-z compilation

Pegando o produto referente ao 2dflens no pzserver:

In [ ]:
specz = pz_server.get_product('52_2dflens_public_specz')

Mostrando os metadados do produto:

In [ ]:
#specz.display_metadata()

Mostrando as estatísticas básicas dos dados:

In [ ]:
specz.data.describe()

Fazendo uma cópia do dataframe para fazer os plots.

In [ ]:
df_2dflens_all = specz.data.copy()

**Plot**

Definindo os pontos a serem utilizados no plot:

In [ ]:
df_input = df_2dflens_all

## Usando apenas uma fração dos dados:
frac = 0.05  
input_sample_for_plots = df_input.sample(frac=frac, axis='index')
assert len(input_sample_for_plots) == round(frac * len(df_input))

## Using all data:
input_sample_for_plots = df_input  ## Comment this line to use a fraction of the data.

print("Total number of objects in the sample: "+str(len(df_input)))
print("Number of objects used in the plots: "+str(len(input_sample_for_plots)))

Redefinindo as coordenadas R.A. e DEC. para os plots:

In [ ]:
coords = SkyCoord(ra=-np.array(input_sample_for_plots.ra)*u.degree, 
                  dec=np.array(input_sample_for_plots.dec)*u.degree, frame='icrs')
input_sample_for_plots.ra = np.array(coords.ra.wrap_at(180*u.degree))
input_sample_for_plots.dec = np.array(coords.dec)

Fazendo o plot com o geoviews e datashader:

In [ ]:
%%time
title = 'Distribuição Espacial - Todos os dados do 2dflens' ### Título do plot.

### Defining the points in geoviews.
ra_dec_points = gv.Points((input_sample_for_plots['ra'],input_sample_for_plots['dec']), kdims=['ra', 'dec'])

### Defining the projected points using cartopy Mollweide().
projected = gv.operation.project(ra_dec_points, projection=crs.Mollweide())

### Using datashader.
dsh_points = dynspread(rasterize(projected).opts(cmap="Viridis", cnorm='log'))
dsh_points = dsh_points.opts(width=width, height=height, padding=padding, title=title, toolbar='above', colorbar=True, tools=['box_select'])

### Plotting.
dsh_points * ra_dec_foot * grid * labels_plot

#### Objetos do 2dflens filtrados para pertencerem ao footprint do DES

A seguir, faremos um filtro para pegar dados dentro do footprint do DES.

**Filtro simples utilizando o pandas**

O filtro a seguir pega objetos com $ra<90$ ou $ra>359$, e $dec < -15$.

In [ ]:
df_2dflens_filt = specz.data.query("ra > 359 | ra < 90. & dec < -15.").copy()

Estatística básica desses dados filtrados:

In [ ]:
df_2dflens_filt.describe()

**Filtro utilizando o geopandas**

Fazendo um polígono com geopandas, que conterá a geometria do footprint do DES.

In [ ]:
! mkdir -p geopandas-des-footprint

In [ ]:
#lat_point_list = foot_df['foot_dec']
#lon_point_list = foot_df['foot_ra']

#polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
#polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])       

#polygon.to_file(filename='./geopandas-des-footprint/polygon.geojson', driver='GeoJSON')
#polygon.to_file(filename='./geopandas-des-footprint/polygon.gpkg', driver="GPKG")
#polygon.to_file(filename='./geopandas-des-footprint/polygon.shp', driver="ESRI Shapefile")

Lendo essa geometria.

In [ ]:
#world = gpd.read_file('./geopandas-des-footprint/polygon.shp')

Transformando o pandas datframe em um geopandas dataframe.

In [ ]:
#gdf = gpd.GeoDataFrame(df_2dflens_all,
#                       geometry=gpd.points_from_xy(df_2dflens_all.ra, 
#                                                   df_2dflens_all.dec)) 

Aplicando o filtro para guardar apenas objetos que estejam dentro do footprint do DES.

In [ ]:
#pip = gdf.within(world.loc[0, 'geometry'])
#df_2dflens_filt = gdf.loc[pip].copy()

**Plot**

Definindo os pontos a serem utilizados no plot:

In [ ]:
df_input = df_2dflens_filt

## Usando apenas uma fração dos dados:
frac = 0.05  
input_sample_for_plots = df_input.sample(frac=frac, axis='index')
assert len(input_sample_for_plots) == round(frac * len(df_input))

## Using all data:
input_sample_for_plots = df_input   ## Comment this line to use a fraction of the data.

print("Total number of objects in the sample: "+str(len(df_input)))
print("Number of objects used in the plots: "+str(len(input_sample_for_plots)))

Redefinindo as coordenadas R.A. e DEC. para os plots:

In [ ]:
coords = SkyCoord(ra=-np.array(input_sample_for_plots.ra)*u.degree, 
                  dec=np.array(input_sample_for_plots.dec)*u.degree, frame='icrs')
input_sample_for_plots.ra = np.array(coords.ra.wrap_at(180*u.degree))
input_sample_for_plots.dec = np.array(coords.dec)

Fazendo o plot dos dados filtrados com o geoviews e datashader:

In [ ]:
%%time
title = 'Distribuição Espacial - Dados do 2dflens filtrados' ### Título do plot.

### Defining the points in geoviews.
ra_dec_points = gv.Points((input_sample_for_plots['ra'],input_sample_for_plots['dec']), kdims=['ra', 'dec'])

### Defining the projected points using cartopy Mollweide().
projected = gv.operation.project(ra_dec_points, projection=crs.Mollweide())

### Using datashader.
dsh_points = dynspread(rasterize(projected).opts(cmap="Viridis", cnorm='log'))
dsh_points = dsh_points.opts(width=width, height=height, padding=padding, title=title, toolbar='above', colorbar=True, tools=['box_select'])

### Plotting.
dsh_points * ra_dec_foot * grid * labels_plot

## Dados fotométricos

### Lendo os dados do DES DR2, contidos na região do 2dflens, com a biblioteca dblinea

Definindo a classe DBBase, que faz a conexão com o banco de dados.

In [ ]:
#db = DBBase()

Definindo quais dados nós queremos acessar. Aqui, nós vamos acessar os dados da tabela **coadd_objects** do catálogo **DES DR2**.

In [ ]:
#schema = "des_dr2"  
#tablename = "coadd_objects"

Obtendo o número de colunas da tabela original (o nome e significado de cada coluna pode ser encontrado [aqui](https://des.ncsa.illinois.edu/releases/dr2/dr2-products/dr2-schema)).

In [ ]:
#tot_columns_list = db.get_table_columns(tablename, schema=schema)
#print("Number of columns in the original table: "+str(len(tot_columns_list)))

Fazendo a busca.

In [ ]:
#extended_class_coadd_lim = 3
#mag_lim_i = 23
#flags_lim_i = 4 

#limit = 300000

In [ ]:
#query = (f"SELECT coadd_object_id, ra, dec, mag_auto_g_dered, mag_auto_r_dered, mag_auto_i_dered, mag_auto_z_dered, mag_auto_y_dered, magerr_auto_g, "+
#         f"magerr_auto_r, magerr_auto_i, magerr_auto_z, magerr_auto_y, flags_g, flags_r, flags_i, flags_z, flags_y, extended_class_coadd "+ 
#         f"FROM des_dr2.coadd_objects "+
#         f"WHERE (dec <= -26.25 AND dec >= -35.73 AND (ra <= 52.75 OR ra>=399) "+
#         f"AND extended_class_coadd >= {extended_class_coadd_lim} "+
#         f"AND mag_auto_i_dered <= {mag_lim_i} "+
#         f"AND flags_i <= {flags_lim_i}) "+
#         f"ORDER BY random() "+
#         f"limit {limit} "   
#        )

In [ ]:
#%%time
#df_phot = db.fetchall_df(query)

### Lendo os dados do DES DR2, contidos na região do 2dflens, salvos localmente

In [ ]:
df_phot = pd.read_csv('des_dr2_RANDOM_limit_300000_ext_class_coadd_3_mag_auto_i_dered_less_eq_23_flags_lim_i_less_eq_4.csv')
df_phot = df_phot.set_index('Unnamed: 0')
df_phot.index.name = None

### Prévia dos dados e estatística básica

Prévia dos dados:

In [ ]:
df_phot.head()

Estatísticas básicas:

In [ ]:
df_phot.describe()

### Plot

Definindo os pontos a serem utilizados no plot:

In [ ]:
df_input = df_phot

## Usando apenas uma fração dos dados:
frac = 0.05  
input_sample_for_plots = df_input.sample(frac=frac, axis='index')
assert len(input_sample_for_plots) == round(frac * len(df_input))

## Using all data:
input_sample_for_plots = df_input   ## Comment this line to use a fraction of the data.

print("Total number of objects in the sample: "+str(len(df_input)))
print("Number of objects used in the plots: "+str(len(input_sample_for_plots)))

Redefinindo as coordendas R.A. e DEC. para os plots:

In [ ]:
coords = SkyCoord(ra=-np.array(input_sample_for_plots.ra)*u.degree, 
                  dec=np.array(input_sample_for_plots.dec)*u.degree, frame='icrs')
input_sample_for_plots.ra = np.array(coords.ra.wrap_at(180*u.degree))
input_sample_for_plots.dec = np.array(coords.dec)

Fazendo o plot dos dados filtrados com o geoviews e datashader:

In [ ]:
%%time
title = 'Distribuição Espacial - Dados do DES DR2 filtrados' ### Título do plot.

### Defining the points in geoviews.
ra_dec_points = gv.Points((-input_sample_for_plots['ra'],input_sample_for_plots['dec']), kdims=['ra', 'dec'])

### Defining the projected points using cartopy Mollweide().
projected = gv.operation.project(ra_dec_points, projection=crs.Mollweide())

### Using datashader.
dsh_points = dynspread(rasterize(projected).opts(cmap="Viridis", cnorm='log'))
dsh_points = dsh_points.opts(width=width, height=height, padding=padding, title=title, toolbar='above', colorbar=True, tools=['box_select'])

### Plotting.
dsh_points * ra_dec_foot * grid * labels_plot

# Utilizando o hipscat_import

## Reescrevendo o arquivo original, com ObjectID fictício (inteiro sequencial)

In [ ]:
### Selecionando as linhas da tabela original utilizando o filtro anterior como máscara.
df_reset = specz.data[specz.data.index.isin(df_2dflens_filt.index)].copy() 

In [ ]:
### Resetando os índices.
df_reset = df_reset.reset_index()
df_reset

In [ ]:
### Salvando o arquivo.
#df_reset.to_parquet('/home/luigi.silva/hipscat_data/specz.pq', index=False)

#df_reset.to_csv('/home/luigi.silva/hipscat_data/specz.csv', index=False)

## Instalação do `hipscat_import` 

```shell
pip install hipscat-import

```




## Execução - conversão dos arquivos para formato Hipscat

https://hipscat-import.readthedocs.io/en/latest/catalogs/public/allwise.html


``` 
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/README.txt
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat.bz2.sizes.txt
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat.bz2.md5.txt
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat-part01.bz2
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat-part02.bz2
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat-part03.bz2 
``` 

In [ ]:
# Load the column names and types from a side file.
#type_frame = pd.read_csv("/home/julia/hipscat_data/pzserver/specz_types.csv")
#type_map = dict(zip(type_frame["name"], type_frame["type"]))

In [ ]:
#type_map

### Catalog Import Arguments

In [ ]:

# args = ImportArguments(
#     output_artifact_name="allwise",
#     input_path="/path/to/allwise/",
#     input_format="csv.bz2",
#     file_reader=CsvReader(
#         header=None,
#         separator="|",
#         column_names=type_frame["name"].values.tolist(),
#         type_map=type_map,
#         chunksize=250_000,
#     ),
#     use_schema_file="allwise_schema.parquet",
#     ra_column="ra",
#     dec_column="dec",
#     sort_columns="source_id",
#     pixel_threshold=1_000_000,
#     highest_healpix_order=7,
#     output_path="/path/to/catalogs/",
# )
# runner.pipeline(args)

In [ ]:
### THIS WAS UNCOMMENTED

#args = ImportArguments(
#    sort_columns="index",
#    ra_column="ra",
#    dec_column="dec",
#    input_path="/home/julia/hipscat_data/pzserver/input",
#    input_format="csv",
#    output_artifact_name="test_cat_specz",
#    output_path="/home/julia/hipscat_data/pzserver/output",
#)

In [ ]:
#runner.pipeline(args)

## Análise dos resultados 

visualização dos dados em formato hipscat (ex: mapa de densidade de pixels)

# Utilizando o LSDB

## Instalação do LSDB


## X-matching usando o LSDB

## Análise dos resultados do X-matching